### Quick Random Forest Model

In [21]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from tqdm import *

test_df = pd.read_csv('Data/encoded_test_df.csv', low_memory=False)
train_df = pd.read_csv('Data/encoded_train_df.csv', low_memory=False)

pd.set_option('max_columns',200)

In [22]:
train_df.shape

(182080, 142)

### DataFrame to Values

In [23]:
train_df.head()

,teacher_id,teacher_prefix,school_state,project_grade_category,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,applied,arts,care,civics,health,history,hunger,language,learning,literacy,math,music,needs,science,special,sports,warmth,cat_applied,cat_arts,cat_care,cat_civics,cat_health,cat_history,cat_hunger,cat_language,cat_learning,cat_literacy,cat_math,cat_music,cat_needs,cat_science,cat_special,cat_sports,cat_warmth,applied.1,arts.1,care.1,career,character,civics.1,college,community,development,esl,early,economics,education,environmental,extracurricular,financial,fitness,foreign,geography,government,gym,health.1,history.1,hunger.1,involvement,languages,life,literacy.1,literature,mathematics,music.1,needs.1,nutrition,other,parent,performing,prep,science.1,sciences,service,social,special.1,sports.1,team,visual,warmth.1,wellness,writing,subcat_applied,subcat_arts,subcat_care,subcat_career,subcat_character,subcat_civics,subcat_college,subcat_community,subcat_development,subcat_esl,subcat_early,subcat_economics,subcat_education,subcat_environmental,subcat_extracurricular,subcat_financial,subcat_fitness,subcat_foreign,subcat_geography,subcat_government,subcat_gym,subcat_health,subcat_history,subcat_hunger,subcat_involvement,subcat_languages,subcat_life,subcat_literacy,subcat_literature,subcat_mathematics,subcat_music,subcat_needs,subcat_nutrition,subcat_other,subcat_parent,subcat_performing,subcat_prep,subcat_science,subcat_sciences,subcat_service,subcat_social,subcat_special,subcat_sports,subcat_team,subcat_visual,subcat_warmth,subcat_wellness,subcat_writing
0,29315,3,33,3,133693,52228,36978,3258,2917,11869,26,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,91061,2,10,0,75471,108379,169951,3258,2917,120270,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,69247,3,44,0,85451,17445,165863,3258,2917,153361,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,33451,1,27,0,161090,88865,140054,3258,2917,74811,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,67811,1,4,1,153420,75662,23554,3258,2917,59211,42,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [90]:
# Convert  pandas valuse to numyp array
y = train_df.project_is_approved.values
X = train_df.loc[:,train_df.columns != 'project_is_approved']
X_test = test_df.loc[:,test_df.columns != 'project_is_approved']

### Model Setup

In [36]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
%%timeit
y_pred = model.predict(X)

1 loop, best of 3: 2.02 s per loop


In [32]:
y_pred = model.predict_proba(X)

### Predicting on the Testing Set

In [91]:
y_pred = model.predict(X_test)
y_pred = model.predict_proba(X_test)

### K-Fold Cross Validation

In [30]:
print('Running')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

crossvalidation = KFold(10, random_state=1)
scores = cross_val_score(model, X, y, 
                scoring = 'accuracy',
                cv = crossvalidation, n_jobs =1)

print ('Folds: %i,accuracy: %.2f std: %.2f' 
% (len(scores),np.mean(np.abs(scores)),np.std(scores)))

Running
Folds: 10,accuracy: 0.83 std: 0.00


In [43]:
ids = pd.read_csv('Data/test.csv', low_memory=False,usecols=['id'])

In [48]:
ids = pd.DataFrame(data=ids); ids.head(5)

,id
0,p233245
1,p096795
2,p236235
3,p233680
4,p171879


In [60]:
test_prediction.head()

,0,1,project_is_approved
0,0.1,0.9,0
1,0.3,0.7,0
2,0.2,0.8,0
3,0.2,0.8,0
4,0.4,0.6,0


In [92]:
ids.shape

(78035, 1)

In [93]:
y_pred.shape

(78035, 2)

In [94]:
test_prediction = pd.DataFrame(data=y_pred)
test_prediction['project_is_approved'] = 0
for index, row in tqdm(test_prediction.iterrows()):
    if test_prediction.loc[index,0] > test_prediction.loc[index,0]:
        test_prediction.loc[index,'project_is_approved'] = test_prediction.loc[index,0]
    else:
        test_prediction.loc[index,'project_is_approved'] = test_prediction.loc[index,1]

78035it [00:45, 1716.93it/s]


In [95]:
sample_submission = pd.concat([ids,test_prediction.project_is_approved], axis=1)

In [97]:
sample_submission.tail(5)

,id,project_is_approved
78030,p183993,0.8
78031,p116343,0.8
78032,p210728,0.8
78033,p060531,0.9
78034,p087783,0.8


In [101]:
sample_submission.to_csv('Submissions/quick_random_forest_model.csv', index=False)